In [2]:
import os

In [3]:
%pwd

'e:\\Deep Learning\\pytorch\\Brain_Tumor_Classification(MRI)\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'e:\\Deep Learning\\pytorch\\Brain_Tumor_Classification(MRI)'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from Brain_Tumor_Classification_MRI.constants import *
from Brain_Tumor_Classification_MRI.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )

          return data_ingestion_config

In [9]:
import os
import zipfile
import gdown
from Brain_Tumor_Classification_MRI import logger
from Brain_Tumor_Classification_MRI.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-04 09:51:57,994: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-04 09:51:58,022: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-04 09:51:58,029: INFO: common: created directory at: artifacts]
[2024-06-04 09:51:58,033: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-04 09:51:58,040: INFO: 599785071: Downloading data from https://drive.google.com/file/d/1JZIida1lwHXR75te-lEd161TTnE0qG_4/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1JZIida1lwHXR75te-lEd161TTnE0qG_4
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1JZIida1lwHXR75te-lEd161TTnE0qG_4&confirm=t&uuid=bd4ec395-e5a3-4354-8c24-0b6c1eb4c9c2
To: e:\Deep Learning\pytorch\Brain_Tumor_Classification(MRI)\artifacts\data_ingestion\data.zip
100%|██████████| 90.9M/90.9M [01:07<00:00, 1.35MB/s]

[2024-06-04 09:53:07,756: INFO: 599785071: Downloaded data from https://drive.google.com/file/d/1JZIida1lwHXR75te-lEd161TTnE0qG_4/view?usp=sharing into file artifacts/data_ingestion/data.zip]
